<a href="https://colab.research.google.com/github/NULabTMN/ps1-AnantShanbhag93/blob/master/PS1_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [0]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np

In [0]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [0]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
from sklearn.feature_extraction.text import CountVectorizer
stopper = {"a","abed","abel","abet","able","about","above","abr","abus","aby","ace","ache","acre","across","act","acts","ad","adam","add","adds","ado","ador","aer","aery","afar","affy","after","afterwards","ag","again","against","agan","agaz","age","aged","ages","ago","ague","ah","aha","ai","aid","aide","aids","ail","aim","aims","aio","air","airs","airy","ajax","al","alas","alb","ale","ales","alit","all","alla","alls","ally","almost","alms","alone","along","alps","already","also","alt","although","always","am","amaz","ambo","amen","ames","amid","amis","among","amongst","amoungst","amount","an","and","anew","ang","angl","anna","anne","anon","another","ant","ante","any","anyhow","anyone","anything","anyway","anywhere","ap","ape","apes","apex","apt","aqua","arc","arch","arcu","arde","are","argo","argu","ark","arm","arma","arme","arms","army","arn","around","art","arts","as","ash","ashy","asia","ask","asks","asp","ass","at","ate","ates","au","auld","aunt","aut","ave","aves","avez","avis","avow","aw","awak","away","awe","awl","awry","axe","axle","ay","aye","ayez","azur","b","ba","baa","babe","baby","bach","back","bad","bade","baes","bag","bags","bail","bait","bak","bake","bal","bald","bale","balk","ball","balm","ban","band","bane","bang","bank","bans","bar","bard","bare","bark","barm","barn","barr","bars","bas","base","bask","bass","bat","bate","bath","bats","batt","bawd","bawl","bay","bays","be","bead","beak","beam","bean","bear","beat","beau","became","because","beck","become","becomes","becoming","bed","beds","bee","beef","been","beer","bees","before","beforehand","beg","begg","begs","behind","being","bel","bell","below","belt","ben","bend","bene","bent","ber","bere","beside","besides","bess","best","bet","bett","between","bevy","bew","beyond","bias","bid","bide","bids","bien","bier","big","bill","bin","bind","bird","bis","bit","bite","bits","bitt","blab","blam","blaz","bled","blew","blo","blot","blow","blue","blur","bo","boar","boat","bob","bobb","bod","bode","bodg","body","bog","bogs","boil","bold","bolt","bon","bona","bond","bone","bood","book","boon","boor","boot","bor","bora","bore","born","boss","both","bots","bottom","bout","bove","bow","bowl","bows","box","boy","boys","brac","brag","bran","bras","brat","brav","bray","braz","bred","brew","brib","brim","brow","buck","bud","buds","buff","bug","bugs","bulk","bull","bum","bump","bums","bung","buoy","bur","burd","burn","burr","burs","bury","bush","buss","busy","but","butt","buy","buys","buzz","by","bye","c","ca","cade","cage","cain","cak","cake","calf","call","calm","cam","came","camp","can","cank","cannot","cant","cap","cape","capp","caps","capt","car","card","care","carl","carp","cars","cart","carv","cas","casa","case","cash","cask","cast","cat","cate","cato","cats","cauf","caus","cave","ce","ceas","cell","ces","cess","chaf","cham","chap","chas","chat","chaw","che","cher","chev","chew","chez","chi","chid","chin","chok","chop","chor","chud","chus","ciel","cine","cite","city","clad","clap","claw","clay","clef","clew","clip","clo","clod","clog","clos","cloy","club","co","coal","coat","cock","cod","cods","cog","coif","coil","coin","col","cold","colt","com","comb","come","con","conn","cons","cony","cook","cool","coop","cop","cope","copp","copy","cor","cord","core","cork","corn","cost","cot","cote","could","couldnt","cour","cow","cowl","cox","coy","coz","crab","cram","crav","craz","crew","crib","crop","crow","crum","cry","cub","cubs","cue","cues","cuff","cull","cum","cup","cups","cur","curb","curd","cure","curl","curs","cut","cuts","cyme","d","dace","dad","daff","dale","dam","dame","damm","damn","damp","dams","dan","danc","dane","dang","dank","dar","dare","dark","dart","dash","dat","date","daub","davy","daw","dawn","daws","day","days","de","dead","deaf","deal","dean","dear","debt","deck","deed","deem","deep","deer","defy","deja","demi","den","deni","dens","deny","deo","dere","dern","des","describe","desk","desp","dest","det","detail","deum","deux","dew","dews","dewy","di","dial","dian","dic","dice","dich","dick","did","dido","die","died","dies","diet","dieu","diff","dig","digg","digs","digt","dild","dim","dimm","dims","din","dine","ding","dint","dion","dip","dipp","dips","dir","dire","dirt","dis","dish","dit","div","dive","dizy","do","dock","doct","doe","doer","does","doff","dog","dogg","dogs","doit","dole","doll","dolt","don","donc","done","dong","donn","doom","door","dost","dote","doth","doug","dout","dove","dow","down","doxy","dozy","drab","drag","dram","draw","dreg","drew","driv","drop","drug","drum","dry","dst","du","dub","dubb","duck","due","duer","dues","duff","dug","dugs","duke","dull","duly","dumb","dump","dun","dung","dupp","during","dust","duty","dy","dye","dyed","e","each","ear","earl","earn","ears","eas","ease","east","easy","eat","eats","eaux","ebb","ebbs","ebon","ecce","eche","echo","ecus","eden","edg","edge","edm","eel","eels","eg","egal","eget","egg","eggs","egma","ego","eie","eight","either","eke","el","elbe","eld","eleven","elf","ell","elle","elm","else","elsewhere","ely","em","empty","en","end","ends","endu","enew","enon","enough","enow","ensu","ent","envy","eo","er","ere","erga","ergo","ern","eros","err","errs","erst","erta","es","espy","est","et","etc","ete","etes","etna","eton","etre","ev","eve","even","ever","every","everyone","everything","everywhere","eves","evil","ew","ewe","ewer","ewes","except","exil","exit","ey","eyas","eye","eyed","eyes","eyne","fa","fac","face","fact","fade","fail","fain","fair","fais","fait","fal","fall","fam","fame","fan","fane","fang","fann","fans","fap","far","fare","farm","fas","fast","fat","fate","fats","faut","fawn","fay","fe","fear","feat","fed","fee","feed","feel","fees","feet","fell","felt","fen","fenc","fens","fer","fere","fery","fest","fet","fett","feu","feud","few","fico","fie","fife","fifteen","fify","fig","figo","figs","fike","fil","file","fill","film","fils","fin","find","fine","fing","finn","fins","fir","fire","firk","firm","first","fish","fist","fit","fits","five","fix","fl","flag","flam","flap","flat","flaw","flax","flay","flea","fled","flee","flew","flow","flux","fly","fo","foal","foam","fob","foe","foes","fog","fogs","foh","foi","foil","foin","fois","foix","fold","folk","fond","font","food","fool","foot","fopp","fops","for","forc","ford","fore","forg","fork","form","former","formerly","fort","forty","fost","foul","found","four","fowl","fox","fram","fran","fray","free","fret","fro","frog","from","front","fry","fubb","fuel","full","fum","fume","fun","fur","furr","further","fury","fust","fut","g","gad","gads","gag","gage","gagg","gain","gait","gale","gall","gam","game","gan","gaol","gap","gape","gaps","gar","garb","gard","gart","gash","gasp","gat","gate","gath","gaud","gaul","gav","gave","gay","gaz","gaze","gear","geck","geld","gelt","gem","gems","gen","gens","gent","gest","get","gets","gi","gib","gibe","gift","gig","gigs","gild","gilt","gin","ging","ginn","gins","gird","girl","girt","gis","giv","give","glad","glaz","glib","glou","glow","gloz","glu","glue","glut","gnat","gnaw","go","goad","goal","goat","god","gods","goer","goes","gogs","gold","gon","gone","gong","good","goot","gor","gore","gorg","gory","goss","got","goth","gout","gown","grac","grav","gray","graz","gree","greg","grew","grey","grim","grin","grip","grop","gros","grow","grub","gud","guil","gul","gulf","gull","gum","gumm","gums","gun","guns","gust","guts","guy","gyve","h","ha","hac","hack","had","haec","hag","hags","hail","hair","hal","hale","half","hall","hals","halt","ham","hams","hand","hang","hap","happ","haps","hard","hare","hark","harm","harp","hart","has","hasnt","hast","hat","hate","hath","hats","haud","hauf","hav","have","hawk","hay","he","head","heal","heap","hear","heat","heav","hedg","heed","heel","heir","held","hell","helm","help","hem","heme","hemm","hemp","hems","hen","hence","hens","hent","her","herb","herd","here","hereafter","hereby","herein","hereupon","hero","hers","herself","hest","hew","hewn","hews","hey","hic","hick","hid","hide","hie","hies","hig","high","hill","hilt","hily","him","himself","hinc","hind","hing","hint","hip","hipp","hips","hir","hire","his","hiss","hist","hit","hits","hive","ho","hoa","hoar","hob","hoc","hod","hog","hogs","hois","hold","hole","holp","holy","home","homo","honi","hoo","hood","hoof","hook","hoop","hoot","hop","hope","hor","horn","hors","hose","host","hot","hour","hous","how","howe","however","howl","hoy","hue","hued","hug","huge","hugg","hugh","hugs","hulk","hull","hum","hume","humh","hums","hundred","hung","hunt","hurl","hurt","hush","hyen","hymn","i","iago","ibat","ice","ici","icy","id","idea","idem","iden","ides","idle","idly","idol","ie","ield","if","ifs","ii","iii","iiii","il","ild","ill","illo","ills","ils","im","imp","in","inc","ince","inch","ind","inde","indeed","indu","ink","inky","inly","inn","inns","inow","int","interest","into","ipse","ira","irae","iras","ire","iris","irks","iron","is","ise","ish","isis","isle","issu","ist","ista","it","itch","item","its","itself","iv","ivy","iwis","ix","j","jack","jade","jail","jamy","jane","jar","jars","jaw","jaws","jay","jays","je","jeer","jest","jesu","jet","jets","jew","jews","jig","jill","joan","job","jog","john","join","jolt","jot","jour","jove","jowl","joy","joys","juan","jud","jude","judg","jug","jugs","jul","jule","july","jump","june","juno","jure","jury","just","kam","kate","keel","keen","keep","ken","kent","kept","kern","key","keys","kibe","kick","kid","kill","kiln","kin","kind","kine","king","kins","kiss","kite","klll","knav","knee","knew","knit","knog","knot","know","l","la","lac","lace","lack","lad","lade","lads","lady","laer","lag","laid","lain","lake","lam","lamb","lame","lamp","lanc","land","lane","lank","lap","lapp","laps","lark","las","lash","lass","last","late","lath","latter","latterly","laud","laur","laus","lave","law","lawn","laws","lay","lays","lazy","ld","ldst","le","lead","leaf","leah","leak","lean","leap","lear","leas","least","leav","led","leda","leek","leer","lees","leet","left","leg","lege","legg","legs","lena","lend","lent","leo","leon","lers","les","less","lest","let","lets","lett","leur","leve","levy","lewd","liar","lick","lid","lids","lie","lied","lief","lien","lies","lieu","life","lift","lig","lik","like","lily","lim","limb","lime","limn","limp","lin","line","ling","link","lion","lip","lipp","lips","lisp","list","liv","live","ll","lnd","lo","loa","load","loaf","loam","loan","lob","lock","lode","lodg","log","logs","lone","long","loo","loof","look","loon","loop","loos","lop","lopp","lord","lorn","lors","los","lose","loss","lost","lot","lots","lott","loud","lour","lout","lov","love","low","lowe","lown","lt","ltd","luc","luce","luck","lucy","lud","lug","lugg","luke","lull","lump","luna","lure","lurk","lush","lust","lute","lux","ly","lye","lyen","lym","lynn","lyon","m","ma","mab","mace","mack","mad","made","mads","mahu","maid","mail","maim","main","mais","mak","make","mal","mala","male","mall","malt","man","mane","mann","mans","many","map","mapp","maps","mar","mare","marg","mark","marl","marr","mars","mart","marv","mary","mas","mask","mass","mast","mate","math","maud","maul","maw","maws","may","maz","maze","me","mead","meal","mean","meanwhile","meat","med","mede","meed","meek","meet","meg","mell","melt","men","mend","mer","mere","mes","mesh","mess","met","mete","meus","mew","mi","mice","mid","might","mihi","mild","mile","milk","mill","milo","min","minc","mind","mine","mint","minx","mio","mir","mire","miry","mis","miss","mist","mite","mix","mo","moan","moat","mock","modo","moe","moi","mole","mome","mon","mong","monk","mons","mood","moon","moor","mop","mope","more","moreover","mori","morn","mort","mose","moss","most","mostly","mote","moth","mots","mous","mov","move","mow","mows","moy","moys","mrs","much","muck","mud","mugs","mule","mull","mum","mun","murd","mure","murk","mus","muse","mush","musk","muss","must","mute","my","myself","myst","n","nag","nage","nags","nail","nak","nal","nam","name","namely","nan","nap","nape","naps","naso","nave","navy","nay","ne","neaf","near","neat","neb","nec","neck","ned","need","neer","neither","nell","ner","nero","ners","nest","net","nets","nev","never","nevertheless","new","news","newt","next","nice","nick","nigh","nile","nill","nine","ning","nip","nipp","nips","nit","nly","no","noah","nob","nobody","nod","nods","noes","nois","nole","non","none","nony","nook","noon","noone","nor","nos","nose","not","note","nothing","noun","nous","novi","now","nowhere","ns","nt","numa","numb","nun","nuns","nurs","nut","nuts","ny","nym","o","oak","oaks","oars","oath","oats","ob","obey","od","odd","odds","ode","odes","ods","oes","of","off","oft","often","oh","oil","oils","oily","old","oman","ome","omen","omit","omne","on","once","one","ones","only","onto","oo","ooze","oozy","op","opal","ope","oped","open","opes","oph","or","orb","orbs","ord","ords","ore","ork","orld","ort","orts","osr","ossa","ost","osw","other","others","otherwise","oui","our","ours","ourselves","out","oven","over","ovid","ow","owe","owed","owen","owes","owl","owls","own","owns","owy","ox","oxen","oyes","p","pac","pace","pack","pad","page","pah","paid","pail","pain","pair","pal","pale","pall","palm","palt","paly","pamp","pan","pang","pant","pap","paps","par","pard","pare","park","part","pas","pash","pass","past","pat","pate","path","paul","pav","paw","pawn","paws","pax","pay","pays","pe","peak","peal","pear","peas","peat","peck","pede","peds","peel","peep","peer","peg","pegs","pelf","pell","pelt","pen","penn","pens","pent","per","pere","perhaps","perk","pers","pert","pest","pet","peto","peu","pew","pia","pick","pid","pie","piec","pied","pier","pies","pig","pike","pil","pile","pilf","pill","pin","pine","pink","pinn","pins","pint","pip","pipe","pisa","pish","piss","pit","pith","pits","pity","piu","pius","plac","play","plea","please","plod","plot","plow","plue","plum","plus","ply","po","pody","poem","poet","pois","poke","pol","pold","pole","poli","poll","pomp","pond","pont","pooh","pool","poop","poor","pop","pope","popp","pops","por","pore","pork","porn","port","pose","post","posy","pot","pots","pour","pout","pow","powd","pox","poys","prat","pray","pre","pret","prey","prie","prig","priz","pro","prop","prov","prun","pry","puck","puff","puft","puis","puke","pull","pump","pun","punk","puny","pur","pure","purg","purr","purs","push","put","puts","py","qu","quae","quak","quam","quat","quay","que","qui","quid","quip","quis","quit","quo","quod","r","race","rack","rafe","raft","rag","rage","ragg","rags","rah","rail","rain","rais","rak","rake","ral","rald","ram","ramm","rams","ran","rang","rank","rant","rap","rape","raps","rapt","rar","rare","rash","rat","rate","rather","rats","rav","rave","raw","ray","rays","raz","raze","re","read","real","reap","rear","reck","red","rede","reed","reek","reel","reft","reg","rein","rel","rely","rend","rent","rer","rere","rers","ress","rest","rets","rex","rey","rful","rhym","rib","ribb","ribs","rice","rich","rid","ride","rids","rien","ries","rift","rig","rigg","ril","rim","rin","rind","ring","riot","rip","ripe","ripp","rise","rish","rite","rive","rivo","rne","road","roam","roan","roar","rob","roba","robb","robe","robs","rock","rod","rode","rods","roe","roes","roi","roll","rom","rome","rood","roof","rook","room","root","rope","ros","rose","ross","rosy","rot","rote","rots","rous","rout","rove","row","roy","rs","rt","rub","rubb","rubs","ruby","rud","rude","rue","rued","ruff","rug","ruin","rul","rule","rump","run","rung","runn","runs","rush","rust","rut","ruth","ry","rye","s","sa","saba","sack","sad","saf","safe","sag","sage","said","sail","sain","sake","sala","sale","sall","salt","salv","same","samp","sand","sans","sap","sara","sard","sat","sate","sauc","sauf","sav","save","saw","saws","say","says","sc","scab","scal","scan","scap","scar","scot","scum","scut","se","sea","seal","seam","sear","seas","seat","sect","sedg","see","seed","seek","seel","seem","seemed","seeming","seems","seen","seer","sees","seiz","seld","self","sell","semi","send","sent","sere","serious","serv","set","sets","sev","several","sew","sex","sh","shag","shak","sham","shap","shar","shav","shaw","she","shed","sher","shes","shew","shin","ship","shod","shoe","shog","shop","shot","should","shov","show","shun","shut","shy","si","sic","sick","side","sies","sift","sigh","sign","silk","sima","simp","sin","since","sincere","sing","sink","sinn","sins","sip","sir","sire","sirs","sit","sith","sits","six","sixty","siz","size","skim","skin","skip","sky","slab","slay","slew","slid","slip","slit","slop","slow","slug","slut","sly","slys","smil","smit","smok","smug","snap","snar","snip","snow","snug","so","soak","soar","sob","sobs","sod","soe","soft","soil","soit","sol","sola","sold","sole","some","somehow","someone","something","sometime","sometimes","somewhere","son","song","sons","sont","soon","sop","sops","sore","sort","sot","soto","sots","soud","soul","sour","sous","sov","sow","sowl","spak","span","spar","spe","sped","spin","spit","spok","spot","spun","spur","spy","st","stab","stag","star","stay","stem","step","stew","still","stir","stog","stol","ston","stop","stor","stow","sty","styl","styx","su","sub","such","suck","sue","sued","sues","suff","sug","suis","suit","sum","summ","sums","sun","sund","sung","sunk","suns","sup","supp","sups","sur","surd","sure","sust","suum","swag","swam","swan","sway","swim","swol","swom","swor","swum","sy","system","t","ta","tack","tag","tah","tail","tak","take","tal","tale","talk","tall","tam","tame","tang","tank","tann","tap","tape","tapp","tar","tarr","tart","task","tat","tatt","tavy","tax","te","team","tear","teat","teem","teen","tell","temp","ten","tend","tens","tent","term","test","text","th","than","that","thaw","the","thee","their","them","themselves","then","thence","ther","there","thereafter","thereby","therefore","therein","thereupon","these","they","thick","thin","third","this","those","thou","though","thre","three","through","throughout","thru","ths","thus","thy","ti","tib","tick","tide","tidy","tie","tied","ties","tiff","tike","til","tile","till","tilt","tim","time","tiny","tip","tipp","tips","tir","tire","tis","tish","to","toad","toby","tock","tod","tods","toe","toes","toge","together","toil","told","toll","tom","tomb","ton","too","took","tool","top","topp","tops","tore","torn","toss","tost","tot","tott","tou","tour","tous","tout","tow","toward","towards","town","toy","toys","trap","tray","tree","tres","trib","trim","trip","trod","trop","trot","trow","troy","true","try","tu","tua","tuae","tub","tubs","tuck","tuft","tug","tugg","tun","tune","tuns","turd","turf","turk","turn","tush","tut","twas","tway","twelve","twenty","twig","twin","twit","two","twos","tyb","tyke","type","tyre","u","udge","uds","ugly","um","un","under","undo","une","until","unto","unus","up","upon","ure","urg","urge","urn","urns","urs","ursa","us","use","used","user","uses","ut","utt","uy","v","va","vail","vain","vais","vale","valu","vane","vant","vara","vary","vast","vat","vaux","ve","veal","vede","veil","vein","vell","veni","vent","ver","verb","vere","verg","vert","very","veux","vex","vi","via","vial","vic","vice","vici","vict","vidi","vie","vied","view","vii","viii","vile","vill","vine","vint","viol","vir","vita","vitx","viva","vive","vivo","viz","voce","voic","void","voke","volt","vor","vore","vos","vot","vour","vous","vow","vows","vox","w","wad","wade","waft","wag","wage","wags","wail","wain","wait","wak","wake","walk","wall","wan","wand","wane","wann","want","war","ward","ware","warm","warn","warp","warr","wars","wart","wary","was","wash","wasp","wast","wat","wav","wave","waw","wawl","wax","way","ways","we","weak","weal","wean","wear","weav","web","wed","wedg","weed","week","ween","weep","weet","well","wend","went","wept","were","wert","west","wet","wgy","what","whatever","whe","when","whence","whenever","where","whereafter","whereas","whereby","wherein","whereupon","wherever","whet","whether","whew","whey","which","while","whin","whip","whit","whither","who","whoa","whoe","whoever","whole","whom","whor","whose","why","wi","wick","wid","wide","wife","wild","will","wilt","win","wind","wine","wing","wink","wins","wip","wipe","wire","wiry","wis","wise","wish","wisp","wist","wit","witb","with","within","without","wits","wiv","wive","wo","woe","woes","wolf","womb","won","wond","wont","woo","wood","woof","wool","woos","wor","word","wore","work","worm","worn","wort","wot","wots","would","wow","wrap","wren","writ","wry","wul","wye","x","xi","xii","xiii","xiv","xv","y","yard","yare","yarn","yaw","yawn","ye","yea","yead","year","yeas","yell","yerk","yes","yet","yew","yok","yoke","yon","yond","york","you","your","yours","yourself","yourselves","zany","zeal","zed","zir","zo","zone"}
def createBasicFeatures(corpus):
  corpus = readShakespeare()
  texts = []
  classes = []
  vocab = []
  for eachdict in corpus:
      texts.append(eachdict['text'])
      classes.append(eachdict['genre'])
  one_hot_vectorizer = CountVectorizer(token_pattern = '\w*[a-zA-Z]\w*')
  one_hot_vectors = one_hot_vectorizer.fit_transform(texts).toarray()
  vocab = one_hot_vectorizer.get_feature_names()
  return one_hot_vectors,classes,vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createInterestingFeatures(corpus):
  #Your code here
  corpus = readShakespeare()
  texts = []
  classes = []
  vocab = []
  for eachdict in corpus:
      texts.append(eachdict['text'])
      classes.append(eachdict['genre'])
#     one_hot_vectorizer = CountVectorizer( analyzer = 'char', ngram_range = (6,6), max_features = 75000)#token_pattern = '\w*[a-zA-Z]\w*' )
#     one_hot_vectorizer = CountVectorizer(ngram_range = (1,2), min_df = 3, max_df=15, max_features = 45000)#token_pattern = '\w*[a-zA-Z]\w*'
  one_hot_vectorizer = CountVectorizer(ngram_range = (1,3), max_features=20000,  stop_words= stopper) #stop_words=stopwords.words('english'))#token_pattern = '\w*[a-zA-Z]\w*'
  one_hot_vectors = one_hot_vectorizer.fit_transform(texts).toarray()
  vocab = one_hot_vectorizer.get_feature_names()
  return one_hot_vectors,classes,vocab


In [0]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  

In [0]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [6]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.615385
The most informative terms for pos are: ['you', 'helena', 'prospero', 'duke', 'sir', 'i', 'leontes', 'a', 'of', 'press', 'president', 'preserving', 'preservers', 'preserved', 'pressed', 'preserve', 'preservative', 'preservation', 'preserv', 'presents']
The most informative terms for neg are: ['him', 's', 'iago', 'imogen', 'o', 'brutus', 'lear', 'ham', 'and', 'what', 'rom', 'president', 'preserving', 'preservers', 'preserver', 'preserved', 'pretense', 'preserve', 'preservative', 'preservation']
----------L2 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['i', 'you', 'duke', 'prospero', 'a', 'helena', 'your', 'antonio', 'sir', 'leontes', 'hermia', 'for', 'lysander', 'ariel', 'sebastian', 'demetrius', 'camillo', 'stephano', 'me', 'parolles']
The most informative terms for neg are: ['iago', 'othello', 's', 'him', 'imogen', 'what', 'lear', 'brutus', 'his', 'cassio', 'o', '

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [7]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.576923
The most informative terms for pos are: ['antonio', 'master', 'helena', 'leontes', 'angelo', 'pedro', 'berowne', 'hermia', 'clown', 'rosalind', 'petruchio', 'valentine', 'syracuse', 'lysander', 'prospero', 'proteus', 'dromio', 'practis', 'power truth', 'powers shall']
The most informative terms for neg are: ['gloucester', 'queen', 'lucius', 'nurse', 'coriolanus', 'macbeth', 'othello', 'troilus', 'caesar', 'imogen', 'timon', 'enter', 'romeo', 'hamlet', 'titus', 'brutus', 'speak', 'praise enter', 'praise perfection', 'powers']
----------L2 Norm-----------
The model's average accuracy is 0.846154
The most informative terms for pos are: ['antonio', 'helena', 'master', 'pedro', 'leontes', 'rosalind', 'clown', 'prospero', 'hermia', 'lysander', 'claudio', 'angelo', 'parolles', 'camillo', 'claud', 'sebastian', 'berowne', 'proteus', 'isabella', 'valentine']
The most informative terms for neg are: ['timon', 'macbeth', 'nurse',